# Auditoria de Sistemas de Vigilância: SINAN vs GAL (Dengue 2026)
**Desenvolvido por:** Álvaro Akama

**Contexto:** Este script realiza o cruzamento entre as notificações administrativas (SINAN) e os exames laboratoriais (GAL) para identificar gargalos no fluxo de dados da Vigilância Epidemiológica do RS.

### Regras de Negócio:
1. **Período de Análise:** 04/01/2026 a 31/01/2026.
2. **Lógica Híbrida:** Confronto entre o Município Notificador (SINAN) e o Município Solicitante (GAL).
3. **Indicador QUALIFICA:** Foco na responsabilidade do serviço de saúde que atendeu o paciente.

In [3]:
import os
import pandas as pd
from datetime import datetime
from dbfread import DBF

# --- PARÂMETROS GLOBAIS ---
DATA_INICIO = '2026-01-04'
DATA_FIM    = '2026-01-31'

# Definição de caminhos para portabilidade
PASTA_INPUT  = '../01_Entrada'
PASTA_OUTPUT = '../02_Saida'
PASTA_DEMO   = '../data_demo'

os.makedirs(PASTA_OUTPUT, exist_ok=True)
os.makedirs(PASTA_DEMO, exist_ok=True)

def carregar_banco(caminho):
    """Função auxiliar para leitura multiformato."""
    ext = os.path.splitext(caminho)[1].lower()
    if ext == '.dbf':
        # Leitura de base legada do SUS
        return pd.DataFrame(iter(DBF(caminho, encoding='latin1', char_decode_errors='ignore')))
    elif ext == '.csv':
        return pd.read_csv(caminho, sep=';', encoding='latin1', low_memory=False)
    return pd.read_excel(caminho) if ext in ['.xlsx', '.xls'] else None


print(f"✅ Ambiente configurado para o intervalo solicitado.")

✅ Ambiente configurado para o intervalo solicitado.


### 1. Extração e Transformação: GAL (Laboratório)
Consolidamos os exames de NS1 e IgM, filtrando pelo município que **solicitou** o exame, garantindo a compatibilidade com os indicadores de gestão.

In [6]:
print("--- Processando bases do GAL ---")
arquivos = os.listdir(PASTA_INPUT)
print(f"🔍 {len(arquivos)} arquivos encontrados na pasta de entrada:")
print(f"{arquivos}")
# Identifica arquivos de exames por palavra-chave
files_gal = [f for f in arquivos if any(x in f.upper() for x in ['IGM', 'NS1']) and f.endswith('.csv')]

dfs_gal = []
for f in files_gal:
    df_t = carregar_banco(os.path.join(PASTA_INPUT, f))
    if df_t is not None:
        # Colunas essenciais para a lógica híbrida
        cols_interesse = ['Data do 1º Sintomas', 'IBGE Município Solicitante', 'IBGE Município de Residência']
        dfs_gal.append(df_t[[c for c in cols_interesse if c in df_t.columns]])

df_gal = pd.concat(dfs_gal, ignore_index=True)
df_gal['Data do 1º Sintomas'] = pd.to_datetime(df_gal['Data do 1º Sintomas'], errors='coerce', dayfirst=True)

# Aplicação do filtro temporal
df_gal = df_gal[(df_gal['Data do 1º Sintomas'] >= DATA_INICIO) & (df_gal['Data do 1º Sintomas'] <= DATA_FIM)]

# Agrupamento Geográfico
resumo_gal_solic = df_gal.groupby('IBGE Município Solicitante').size().reset_index(name='Qtd_GAL_Solicitante')
resumo_gal_resid = df_gal.groupby('IBGE Município de Residência').size().reset_index(name='Qtd_GAL_Residencia')

# Normalização do Código IBGE para 6 dígitos
for df in [resumo_gal_solic, resumo_gal_resid]:
    col_ibge = [c for c in df.columns if 'IBGE' in c][0]
    df['COD_IBGE'] = df[col_ibge].astype(str).str.replace(r'\.0$', '', regex=True)

print(f"✅ Dados do GAL consolidados.")

--- Processando bases do GAL ---
🔍 4 arquivos encontrados na pasta de entrada:
['DENGON2026.dbf', 'igm_imunoensaio.csv', 'Municipios_RS.xlsx', 'ns1_imunoensaio.csv']
✅ Dados do GAL consolidados.


### 2. Extração e Transformação: SINAN (Notificações)
Focamos no campo `ID_MUNICIP`, que representa o município que realizou a notificação no sistema.

In [35]:
print("--- Processando base do SINAN ---")
file_sinan = next((f for f in arquivos if 'DENGON' in f.upper() and f.endswith('.dbf')), None)

if file_sinan:
    df_sinan = carregar_banco(os.path.join(PASTA_INPUT, file_sinan))
    df_sinan['DT_SIN_PRI'] = pd.to_datetime(df_sinan['DT_SIN_PRI'], errors='coerce')
    
    # Filtro temporal
    df_sinan = df_sinan[(df_sinan['DT_SIN_PRI'] >= DATA_INICIO) & (df_sinan['DT_SIN_PRI'] <= DATA_FIM)]
    
    # Contagem por Município Notificador
    resumo_sinan = df_sinan.groupby('ID_MUNICIP').size().reset_index(name='Qtd_SINAN_Notificador')
    resumo_sinan['COD_IBGE'] = resumo_sinan['ID_MUNICIP'].astype(str).str.replace(r'\.0$', '', regex=True)
    
    print(f"✅ Dados do SINAN consolidados.")

--- Processando base do SINAN ---
✅ Dados do SINAN consolidados.


### 3. Integração e Geração de Alertas
Cruzamos as bases e geramos o status de conformidade. 
**Nota:** Municípios com exames no GAL mas sem notificações no SINAN recebem o status de **🚨 ALERTA**.

In [36]:
# Cruzamento com Dicionário Oficial de Municípios do RS
file_mun = next((f for f in arquivos if 'MUNIC' in f.upper()), None)
df_mun = carregar_banco(os.path.join(PASTA_INPUT, file_mun))
df_mun['COD_IBGE'] = df_mun['Ibge_6'].astype(str)

# Integração Final (Híbrida)
df_final = pd.merge(df_mun[['COD_IBGE', 'Nome_Municipio', 'CRS']], resumo_sinan[['COD_IBGE', 'Qtd_SINAN_Notificador']], on='COD_IBGE', how='left')
df_final = pd.merge(df_final, resumo_gal_solic[['COD_IBGE', 'Qtd_GAL_Solicitante']], on='COD_IBGE', how='left')
df_final = pd.merge(df_final, resumo_gal_resid[['COD_IBGE', 'Qtd_GAL_Residencia']], on='COD_IBGE', how='left')

# Tratamento de valores nulos e tipagem (Correção dos decimais)
cols_contagem = ['Qtd_SINAN_Notificador', 'Qtd_GAL_Solicitante', 'Qtd_GAL_Residencia']
df_final[cols_contagem] = df_final[cols_contagem].fillna(0).astype(int)

# Definição do Status de Auditoria
def avaliar_status(row):
    if row['Qtd_GAL_Solicitante'] > 0 and row['Qtd_SINAN_Notificador'] == 0:
        return "🚨 ALERTA: GAL sem SINAN"
    elif row['Qtd_SINAN_Notificador'] > 0 and row['Qtd_GAL_Solicitante'] == 0:
        return "❓ SINAN sem GAL"
    return "✅ OK" if row['Qtd_SINAN_Notificador'] > 0 else "Sem atividade"

df_final['STATUS_AUDITORIA'] = df_final.apply(avaliar_status, axis=1)

# --- EXPORTAÇÃO ---
# 1. Relatório Interno (Excel)
df_final.to_excel(os.path.join(PASTA_OUTPUT, f'Monitoramento_CEVS_{DATA_FIM}.xlsx'), index=False)

# 2. Base de Demonstração (GitHub - Anonimizada)
df_final.to_csv(os.path.join(PASTA_DEMO, 'monitoramento_demo.csv'), sep=';', encoding='utf-8-sig', index=False)

print(f"🚀 Processo concluído. Arquivos gerados com sucesso.")

🚀 Processo concluído. Arquivos gerados com sucesso.
